In [1]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming the functions are HTTP endpoints
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/download_files'
function2_url = 'https://europe-west3-lumpito.cloudfunctions.net/extract_table'
function3_url = 'https://europe-west3-lumpito.cloudfunctions.net/standardize_data'
company_ids = [1,2,3,4,5,6,7,8]  # Your list of 14,000 company_ids

def call_function_in_sequence(company_id):
    session = requests.Session()  # Reuse connections within the function
    json={"company_id": company_id}
    # First function call
    response1 = session.post(function1_url, json=json)
    if response1.status_code != 200:
        return company_id, False, 'Function 1 failed'

    # Second function call
    response2 = session.post(function2_url, json=json)
    if response2.status_code != 200:
        return company_id, False, 'Function 2 failed'

    # Third function call
    response3 = session.post(function3_url, json=json)
    if response3.status_code != 200:
        return company_id, False, 'Function 3 failed'

    return company_id, True, 'Success'

# The number of workers can be adjusted based on your system and network capabilities
workers = 10 

with ThreadPoolExecutor(max_workers=workers) as executor:
    # Dictionary to keep track of futures for each company_id
    future_to_company_id = {executor.submit(call_function_in_sequence, cid): cid for cid in company_ids}

    for future in as_completed(future_to_company_id):
        company_id = future_to_company_id[future]
        try:
            _, success, message = future.result()
            if success:
                print(f"Company ID {company_id}: All functions succeeded")
            else:
                print(f"Company ID {company_id}: {message}")
        except Exception as exc:
            print(f"Company ID {company_id} generated an exception: {exc}")

Company ID 8: Function 1 failed
Company ID 1: Function 1 failed
Company ID 5: Function 1 failed
Company ID 7: Function 1 failed
Company ID 6: Function 1 failed
Company ID 2: Function 1 failed
Company ID 4: Function 1 failed
Company ID 3: Function 1 failed


In [3]:
session = requests.Session() 
json={"company_id": company_id}
function1_url = 'https://europe-west3-lumpito.cloudfunctions.net/download_files'
response1 = session.post(function1_url, json=json)
print(response1.text)

Bad Request: Please provide a company name and document types
